In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import os
import glob

import json
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface

In [2]:
# use glob to get all the csv files 
# in the folder
path = os.path.join(os.getcwd(), "crime_data")
csv_files = glob.glob(os.path.join(path, "crime-incident-reports*.csv"))

crime_data = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for f in csv_files:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(f, index_col=None, low_memory=False, encoding = "ISO-8859-1")
    content.append(df)
  
# converting content to data frame
crime_data = pd.concat(content, ignore_index=True)
crime_data.to_csv('crime_data.csv' , index=False, encoding = "ISO-8859-1")

ValueError: No objects to concatenate

In [ ]:
# Get Neigboorhood creating 

In [ ]:
location = crime_data[['Lat', 'Long']].drop_duplicates()


In [ ]:
# I'm checking if there is any values that can not be coordinate
location.dropna().astype('int').drop_duplicates()

In [ ]:
# we need to remove NA's -1 and 0 as they can not be real coordinates. so I will remove anything less than 0
location = location[~(location['Lat'] <= 1)].dropna()

In [ ]:
# I'm checking if there is any values left that can not be coordinate
location.dropna().astype('int').drop_duplicates()

In [ ]:
# Reading neighborhood geojson
with open("Boston_Neighborhoods.geojson") as f:
  features = json.load(f)["features"]

#defining function to find for finding neighborhood name
def find_neighborhood(longitude,latitude):
    try:
        point_to_check =(longitude,latitude)
        for i in range(len(features)):
            boundary = features[i]['geometry'] # get a boundary polygon
            if Point(point_to_check).within(shape(boundary)): # make a point and see if it's in the polygon
               name = features[i]['properties']['Name'] # get the second field of the corresponding record
               return name
    except:
        return

In [ ]:
# Applying function and creating neighborhood column
location['neighborhood']= np.vectorize(find_neighborhood)(location['Long'], location['Lat'])

In [ ]:
crime_data_w_neighborhood = pd.merge(crime_data, location, on=['Lat', 'Long'])

In [ ]:
crime_data_w_neighborhood.to_csv('crime_data_w_neighborhood.csv' , index=False, encoding = "ISO-8859-1")

In [ ]:
crime_data_w_neighborhood_all = pd.merge(crime_data, location, on=['Lat', 'Long'], how='left')

In [ ]:
crime_data_w_neighborhood_all.to_csv('crime_data_w_neighborhood_all.csv' , index=False, encoding = "ISO-8859-1")